In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
import openpyxl
from openpyxl import load_workbook

import Moduli.ricatalogazioni as r
import Moduli.trittico_string_cleaner as c

import os

cwd = os.getcwd().rpartition("\\")[0]

file_categorie = cwd + r"\Parametriche\categorie.xlsx"
data_exp_fibra = cwd + r"\Dashboard performance campaigns\Data export\GIORNALIERO_inclusa_FWA.csv"
data_exp_opzioni = cwd + r"\Dashboard performance campaigns\Data export\GIORNALIERO_OPZIONI.csv"
data_exp_mobile = cwd + r"\Dashboard performance campaigns\Data export\TIM_ECM_thank_you_Offerta_Mobile.csv"

#################### LETTURA FILE PARAMETRICHE ####################

categorie_xlsx = pd.ExcelFile(file_categorie)
p_categorie = pd.read_excel(categorie_xlsx, 'categorie')

#################### Lettura data dell'ultimo file ####################

from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir("C:/Users/11622901/OneDrive - TIM/Team performance/Reports/Dashboard performance campaigns/Source/Fibra") if isfile(join("C:/Users/11622901/OneDrive - TIM/Team performance/Reports/Dashboard performance campaigns/Source/Fibra", f))]

lastFile = onlyfiles[-1]
lastDatetime = pd.to_datetime(lastFile[6:16], format='%Y_%m_%d') + pd.Timedelta(1, unit="Day")

#################### LETTURA DATA EXPORT ####################

### Variabili
d_parser = lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
columns = ["Timestamp","Campaign","CampaignType","Media","Placement","Banner/Adgroup","Ad Interaction","No Repeats","SubOrder","Order ID","Offerta","ID Offerta","Tipologia Offerta"]


### ETL "GIORNALIERO_inclusa_FWA"
af_fibra = pd.read_csv(data_exp_fibra, sep=None, engine="python", encoding = "utf16", parse_dates=["Timestamp"], date_parser=d_parser, usecols=columns)

filt = (af_fibra["No Repeats"] == "Yes") & (af_fibra["SubOrder"] == "No") & (af_fibra["Timestamp"] > lastDatetime) & (af_fibra["Timestamp"].dt.date < dt.date.today())
af_fibra = af_fibra[filt]

af_fibra.drop_duplicates(subset="Order ID", keep="last", inplace= True)

af_fibra["Timestamp"] = af_fibra["Timestamp"].dt.date
af_fibra["Attivazioni"] = 1

af_fibra.fillna("", inplace=True)
af_fibra["Key"] = af_fibra[["Campaign", "Placement", "Banner/Adgroup"]].apply(lambda x: '_'.join(x), axis=1)
af_fibra['Key'] = af_fibra['Key'].str.translate(c.trans_table).apply(lambda x: c.remov_duplicates(x))
af_fibra['Key'] = af_fibra['Key'].apply(lambda x: '_'.join([item for item in x.split("_") if item not in c.stopwords]))



### ETL "GIORNALIERO_OPZIONI"
af_opzioni = pd.read_csv(data_exp_opzioni, sep=None, engine="python", encoding = "utf16", parse_dates=["Timestamp"], date_parser=d_parser, usecols=columns)

filt = (af_opzioni["No Repeats"] == "Yes") & (af_opzioni["SubOrder"] == "No") & (af_opzioni["Timestamp"] > lastDatetime) & (af_opzioni["Timestamp"].dt.date < dt.date.today())
af_opzioni = af_opzioni[filt]

af_opzioni.drop_duplicates(subset="Order ID", keep="last", inplace= True)

af_opzioni["Timestamp"] = af_opzioni["Timestamp"].dt.date
af_opzioni["Attivazioni"] = 1

af_opzioni.fillna("", inplace=True)
af_opzioni["Key"] = af_opzioni[["Campaign", "Placement", "Banner/Adgroup"]].apply(lambda x: '_'.join(x), axis=1)
af_opzioni['Key'] = af_opzioni['Key'].str.translate(c.trans_table).apply(lambda x: c.remov_duplicates(x))
af_opzioni['Key'] = af_opzioni['Key'].apply(lambda x: '_'.join([item for item in x.split("_") if item not in c.stopwords]))



### ETL "TIM_ECM_thank_you_Offerta_Mobile"
af_mobile = pd.read_csv(data_exp_mobile, sep=None, engine="python", encoding = "utf16", parse_dates=["Timestamp"], date_parser=d_parser, usecols=columns)

filt = (af_mobile["No Repeats"] == "Yes") & (af_mobile["SubOrder"] == "No") & (af_mobile["Timestamp"] > lastDatetime) & (af_mobile["Timestamp"].dt.date < dt.date.today())
af_mobile = af_mobile[filt]

af_mobile.drop_duplicates(subset="Order ID", keep="last", inplace= True)

af_mobile["Timestamp"] = af_mobile["Timestamp"].dt.date
af_mobile["Attivazioni"] = 1

af_mobile.fillna("", inplace=True)
af_mobile["Key"] = af_mobile[["Campaign", "Placement", "Banner/Adgroup"]].apply(lambda x: '_'.join(x), axis=1)
af_mobile['Key'] = af_mobile['Key'].str.translate(c.trans_table).apply(lambda x: c.remov_duplicates(x))
af_mobile['Key'] = af_mobile['Key'].apply(lambda x: '_'.join([item for item in x.split("_") if item not in c.stopwords]))



### CONCATENAZIONE DATAFRAMES
frames = [af_fibra, af_opzioni, af_mobile]
dataframes = pd.concat(frames, axis=0, ignore_index=True)



#################### OUTPUT DATI ####################

mese = dataframes["Timestamp"].max().strftime("%m")
anno = dataframes["Timestamp"].max().strftime("%Y")
giorno = dataframes["Timestamp"].max().strftime("%d")


af_fibra.loc[:,['Timestamp', 'Campaign', 'CampaignType', 'Media', 'Placement','Banner/Adgroup', 'Ad Interaction', 'Order ID', 'Offerta', 'ID Offerta', 'Tipologia Offerta', 'Attivazioni', 'Key']].to_excel(cwd + f"/Dashboard performance campaigns/Source/Fibra/fibra_{anno}_{mese}_{giorno}.xlsx", index=False)

af_opzioni.loc[:,['Timestamp', 'Campaign', 'CampaignType', 'Media', 'Placement','Banner/Adgroup', 'Ad Interaction', 'Order ID', 'Offerta', 'ID Offerta', 'Tipologia Offerta', 'Attivazioni', 'Key']].to_excel(cwd + f"/Dashboard performance campaigns/Source/Opzioni/opzioni_{anno}_{mese}_{giorno}.xlsx", index=False)

af_mobile.loc[:,['Timestamp', 'Campaign', 'CampaignType', 'Media', 'Placement','Banner/Adgroup', 'Ad Interaction', 'Order ID', 'Offerta', 'ID Offerta', 'Tipologia Offerta', 'Attivazioni', 'Key']].to_excel(cwd + f"/Dashboard performance campaigns/Source/Mobile/mobile_{anno}_{mese}_{giorno}.xlsx", index=False)



#################### EDIT FILE CATEGORIE ####################

for key, value in r.ricatalogazioni.items():

    conditions = []
    values = []

    for key_inner, value_inner in value.items():

        if len(value_inner) == 1:
            conditions.append(dataframes["Key"].str.contains(value_inner[0]))

        else:
            conditions.append(dataframes["Key"].str.contains('|'.join(value_inner)))

        values.append(key_inner)


    dataframes[key] = np.select(conditions, values)


# CATEGORIE
s_categorie = dataframes.loc[:,["Key", "Tipologia Campagna", "Canale", "Prodotto", "Obiettivo"]]

s_categorie.sort_values(by="Key",inplace=True)
s_categorie.drop_duplicates(subset="Key", keep="first",inplace= True)
s_categorie.fillna("", inplace=True)

categorie = pd.concat([p_categorie,s_categorie], axis=0, ignore_index=True)
categorie.drop_duplicates(subset="Key", keep="first", inplace= True)



#################### MODIFICA FILE PARAMETRICHE ####################

book = load_workbook(file_categorie)

writer = pd.ExcelWriter(file_categorie, engine='openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

book.worksheets[0].tables['categorie'].ref = 'A1:{}{}'.format(chr(len(categorie.columns)+64), len(categorie)+1)

categorie.to_excel(writer, sheet_name='categorie', startrow=0, index=False)

writer.save()
